# Percolación Ejercicio 1
***
## Grupo:
1. Adrián Bedón
1. Merlo José Miguel
1. Moreta Andrés
1. Ocaña Dennis
1. Ramos Xavier
***

## Qué es la percolación?
Percolación hace referencia al paso de fluidos a travez de materiales porosos. Un ejemplo de percolación es la filtracion<br>
<br>
<div>
<img src="https://cdn0.ecologiaverde.com/es/posts/3/2/1/como_hacer_un_filtro_de_agua_casero_para_beber_1123_orig.jpg" width="500"/>
</div>

***

## Simulación de Montecarlo para estudiar un fenómeno natural conocido como percolación

Para el proposito de estas simulaciones se definirá un sistema de `n * n` a modo de cuaadrícula. Dentro de esta cada cuadrante puede estar bloqueado o disponible y los cuadrantes disponibles son inicializados _vacíos_. Un sitio _lleno_ es un sitio disponible que se conecta a un sitio aledaño que también está _diponible_ siendo estos: arriba, abajo, izquierda y/o derecha).<br>
<br>
El sistema se dirá que percola si es que hay un sitio disponible _completo_ esto significando que esta conectado de un extremo a otro de la cuadrícula.<br>
<br>
<div>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-yes.png" width="300"/>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-no.png" width="300"/>
</div>

***

## Percolación vertical
Dada una matriz que representa los sitios disponibles, debemos determinar si este sistema percola. Empezando com percolación vertical siendo este el más simple ya que solo buscará percolación directamente en el prano vertical
<br>
<br>
<div>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-vertically-yes.png" width="300"/>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-vertically-no.png" width="300"/>
</div>

***

In [1]:
## Importamos librerias necesarias
#import numpy as np
import numpy as np #importamos numpy para utilizar linspace
import numba as nb #se importa numba para aceleracion por GPU en diversas funciones
from numba import cuda #para aceleracion por GPU en diversas funciones
from numba import jit #para aceleracion por GPU en diversas funciones
from numba import njit #para aceleracion por GPU en diversas funciones

In [2]:
## Arma un array segun las probabilidades que se indique para las celdas disponibles
@jit
def armarArray(n, probabilidadDisponible):
    array = np.random.binomial(1, probabilidadDisponible, size=(n,n))
    return array

In [3]:
@jit
def flowVertical(array):
    n = len(array)
    ArrayCamino = np.zeros((n,n),dtype=int)
    for j in range(n):
        _flowVertical(array, ArrayCamino, 0, j)
    return ArrayCamino

In [4]:
## Arma un funcion recursiva para armar un array aparte que solo tendra los registros de las
## celdas disponibles que esten conectadas entre si
@jit
def _flowVertical(isOpen, isFull, i, j):
    n = len(isFull)
    if (i < 0) or (i >= n):
        return
    if (j < 0) or (j >= n):
        return
    if (isOpen[i][j] == 0):
        return
    if (isFull[i][j] == 1):
        return
    isFull[i][j] = 1
    _flowVertical(isOpen, isFull, i+1, j)  ## Abajo

In [5]:
## Ejecuta los segmentos recursivos del codigo y detecta si hay un 1 en la fila final
## ya que al haberlo, significa que si percola
@jit
def percolaVertical(isOpen):
    
    ## crea la matriz de 1 que estan juntos
    isFull = flowVertical(isOpen)
    
    
    ## verifica si hay algun 1 en la fila final ya que si es asi, el sistema si percola
    n = len(isFull)
    for j in range(n):
        if (isFull[n-1][j]):
            return True
    return False


In [42]:
## definimos una funcion que tomara n siendo esta la dimension de la matriz
## y p siendo la probabilidad de que la celda este disponible
## esta version incluye un print por propositos de demostración

def mainVerticalPrint(n,p):
    array = armarArray(n,p)
    print(array)
    print(percolaVertical(array))

## Ejemplo con un 2x2
mainVerticalPrint(2,0.5)

## Ejemplo con un 3x3
mainVerticalPrint(3,0.5)

## Ejemplo con un 4x4
mainVerticalPrint(4,0.5)

[[1 1]
 [1 0]]
True
[[1 0 0]
 [0 0 0]
 [0 1 0]]
False
[[1 0 0 0]
 [1 1 0 0]
 [0 0 1 1]
 [0 1 1 0]]
False


***

### Ejemplos para percolacion vertical con 𝑝∈{0.1,0.5,0.9}. 

In [7]:
## definimos una funcion que tomara n siendo esta la dimension de la matriz
## y p siendo la probabilidad de que la celda este disponible

def mainVertical(n,p):
    array = armarArray(n,p)
    return percolaVertical(array)

In [8]:
## para 0.1
j=10000000
contador=0
p=0.1
n=2
for i in range (j):
    if mainVertical(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e1=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e1 ,"%")

Percoló 198765 veces
Esto resulta en una probabilidad de 1.98765 %
Comparado con la probabilidad por fórmula la cual es 1.9900000000000004 %
Para este ejemplo existió un error de: -0.11809045226133257 %


In [12]:
## para 0.5

j=10000000
contador=0
p=0.5
n=2
for i in range (j):
    if mainVertical(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e5=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e5,"%")

Percoló 4376138 veces
Esto resulta en una probabilidad de 43.76138 %
Comparado con la probabilidad por fórmula la cual es 43.75 %
Para este ejemplo existió un error de: 0.026011428571434538 %


In [16]:
## para 0.9

j=10000000
contador=0
p=0.9
n=2
for i in range (j):
    if mainVertical(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e9=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e9,"%")

Percoló 9639214 veces
Esto resulta en una probabilidad de 96.39214 %
Comparado con la probabilidad por fórmula la cual es 96.39 %
Para este ejemplo existió un error de: 0.002220147318183571 %


Viendo la similitud de los valores, se puede decir que la fórmula de probabilidad de percolación es correcta para el sistema de 2x2. Si bien difieren los resultados experimentales de los resultados analíticos, estos son los suficientemente cercanos con un error promedio entre las 10 ejecuciones de:

In [17]:
ErrorP=(e1+e2+e3+e4+e5+e6+e7+e8+e9)
print((ErrorP/9),"%")

-0.016970710216073107 %


Y con esto se puede concluir que la formula _(p**2*(2-p**2))_ funciona para el cálculo de la probabilidad de percolación para una cuadricula de 2x2

***

## Percolación regular

Dada una matriz que representa los sitios disponibles, debemos determinar si este sistema percola. En los siguientes ejemplos se considera izquierda y derecha para el fluir de la percolación.

In [24]:
## Arma un funcion recursiva para armar un array aparte que solo tendra los registros de las
## celdas disponibles que esten conectadas entre si
@jit
def _flow(isOpen, isFull, i, j):
    n = len(isFull)
    if (i < 0) or (i >= n):
        return
    if (j < 0) or (j >= n):
        return
    if (isOpen[i][j] == 0):
        return
    if (isFull[i][j] == 1):
        return
    isFull[i][j] = 1
    _flow(isOpen, isFull, i+1, j  )  ## Abajo
    _flow(isOpen, isFull, i  , j+1)  ## Derecha
    _flow(isOpen, isFull, i  , j-1)  ## Izquierda
    _flow(isOpen, isFull, i-1, j  )  ## Arriba

In [25]:
## Crea un array nuevo en el cual se guardaran las celdas disponibles que esten conectadas
## a las demas y las detecta
@jit
def flow(isOpen):
    n = len(isOpen)
    isFull = np.zeros((n,n),dtype=int)
    for j in range(n):
        _flow(isOpen, isFull, 0, j)
    return isFull

In [26]:
## Ejecuta los segmentos recursivos del codigo y detecta si hay un 1 en la fila final
## ya que al haberlo, significa que si percola
@jit
def percola(isOpen):
    
    ## crea la matriz de 1 que estan juntos
    isFull = flow(isOpen)
    
    
    ## verifica si hay algun 1 en la fila final ya que si es asi, el sistema si percola
    n = len(isFull)
    for j in range(n):
        if (isFull[n-1][j]):
            return True
    return False


In [27]:
## Arma un funcion recursiva para armar un array aparte que solo tendra los registros de las
## celdas disponibles que esten conectadas entre si
@jit
def _flow(isOpen, isFull, i, j):
    n = len(isFull)
    if (i < 0) or (i >= n):
        return
    if (j < 0) or (j >= n):
        return
    if (isOpen[i][j] == 0):
        return
    if (isFull[i][j] == 1):
        return
    isFull[i][j] = 1
    _flow(isOpen, isFull, i+1, j  )  ## Abajo
    _flow(isOpen, isFull, i  , j+1)  ## Derecha
    _flow(isOpen, isFull, i  , j-1)  ## Izquierda
    _flow(isOpen, isFull, i-1, j  )  ## Arriba

In [28]:
## definimos una funcion que tomara n siendo esta la dimension de la matriz
## y p siendo la probabilidad de que la celda este disponible
## esta version incluye un print por propositos de demostración
def mainP(n, probabilidadDisponible):
    array = armarArray(n, probabilidadDisponible)
    print(array)
    print(percola(array))

In [1]:
## Ejecuta el metodo main que toma las dimensiones n*n de la matriz 
## y un numero float entre 0 y 1 el cual representara la 
## probabilidad de asumir 1 o disponible
## Ejemplo con un 2x2
mainP(2, 0.5)
## Ejemplo con un 3x3
mainP(3, 0.5)
## Ejemplo con un 4x4
mainP(4, 0.5)

NameError: name 'mainP' is not defined

### Pruebas para percolacion regular con 𝑝∈{0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9}. 

Se realiza pruebas con un segundo algoritmo que es capaz de detectar percolacion horizontal y vertical, a pesar de que en un sistema de percolación de 2x2 solo existe percolación de tipo vertical

In [30]:
## definimos una funcion que tomara n siendo esta la dimension de la matriz
## y p siendo la probabilidad de que la celda este disponible
def main(n, probabilidadDisponible):
    array = armarArray(n, probabilidadDisponible)
    return (percola(array))

In [31]:
## para 0.1 y un sistema 2x2 
j=10000000
contador=0
p=0.1
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e1=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e1,"%")

Percoló 198928 veces
Esto resulta en una probabilidad de 1.98928 %
Comparado con la probabilidad por fórmula la cual es 1.9900000000000004 %
Para este ejemplo existió un error de: -0.036180904522638085 %


In [32]:
## para 0.2 y un sistema 2x2 
j=10000000
contador=0
p=0.2
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e2=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e2,"%")

Percoló 784173 veces
Esto resulta en una probabilidad de 7.841729999999999 %
Comparado con la probabilidad por fórmula la cual es 7.840000000000001 %
Para este ejemplo existió un error de: 0.022066326530592536 %


In [33]:
## para 0.3 y un sistema 2x2 
j=10000000
contador=0
p=0.3
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e3=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e3,"%")

Percoló 1716805 veces
Esto resulta en una probabilidad de 17.16805 %
Comparado con la probabilidad por fórmula la cual es 17.19 %
Para este ejemplo existió un error de: -0.12769051774287585 %


In [34]:
## para 0.4 y un sistema 2x2 
j=10000000
contador=0
p=0.4
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e4=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e4,"%")

Percoló 2942144 veces
Esto resulta en una probabilidad de 29.421439999999997 %
Comparado con la probabilidad por fórmula la cual es 29.440000000000005 %
Para este ejemplo existió un error de: -0.0630434782608964 %


In [35]:
## para 0.5 y un sistema 2x2 
j=10000000
contador=0
p=0.5
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e5=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e5,"%")

Percoló 4373785 veces
Esto resulta en una probabilidad de 43.73785 %
Comparado con la probabilidad por fórmula la cual es 43.75 %
Para este ejemplo existió un error de: -0.027771428571424756 %


In [36]:
## para 0.6 y un sistema 2x2 
j=10000000
contador=0
p=0.6
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e6=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e6,"%")

Percoló 5902282 veces
Esto resulta en una probabilidad de 59.022819999999996 %
Comparado con la probabilidad por fórmula la cual es 59.040000000000006 %
Para este ejemplo existió un error de: -0.029098915989177517 %


In [37]:
## para 0.7 y un sistema 2x2 
j=10000000
contador=0
p=0.7
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e7=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e7,"%")

Percoló 7397957 veces
Esto resulta en una probabilidad de 73.97957 %
Comparado con la probabilidad por fórmula la cual es 73.99 %
Para este ejemplo existió un error de: -0.014096499526962422 %


In [38]:
## para 0.8 y un sistema 2x2 
j=10000000
contador=0
p=0.8
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e8=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e8,"%")

Percoló 8703762 veces
Esto resulta en una probabilidad de 87.03762 %
Comparado con la probabilidad por fórmula la cual es 87.04 %
Para este ejemplo existió un error de: -0.0027343750000026067 %


In [39]:
## para 0.9 y un sistema 2x2 
j=10000000
contador=0
p=0.9
n=2
for i in range (j):
    if main(n,p):
        contador=contador+1
        
PAnalitica=(p**n*(n-p**n))*100
PExperim=(contador/j)*100
e9=(((PExperim)-PAnalitica)/(PAnalitica))*100
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", PExperim,"%")
print("Comparado con la probabilidad por fórmula la cual es", PAnalitica,"%")
print("Para este ejemplo existió un error de:" , e9,"%")

Percoló 9638831 veces
Esto resulta en una probabilidad de 96.38831 %
Comparado con la probabilidad por fórmula la cual es 96.39 %
Para este ejemplo existió un error de: -0.0017532939101529378 %


In [40]:
ErrorP=(e1+e2+e3+e4+e5+e6+e7+e8+e9)
print((ErrorP/9),"%")

-0.031144787443726445 %
